In [ ]:
import pymongo

# Create the sharded cluster

In [ ]:
cli_config = pymongo.MongoClient(port=27019)

In [ ]:
cli_config.admin.command('replSetInitiate')  # rs.initiate()

In [ ]:
cli_mongos = pymongo.MongoClient(port=27117)

In [ ]:
cli_mongos.admin.command('addShard', 'mongod-data:27018')

In [ ]:
cli_mongos.config.collection_names()

In [ ]:
list(cli_mongos.config.mongos.find())  # sh.status()

# Shard a collection for routeability

In [ ]:
cli_mongos.drop_database('test')
db = cli_mongos.test
posts = db.posts
comments = db.comments

In [ ]:
def make_post(title, author, text):
    result = posts.insert_one({
        'title': title,
        'author': author,
        'text': text
    })
    return result.inserted_id

def make_comment(post_id, author, text):
    result = comments.insert_one({
        'post_id': post_id,
        'author': author,
        'text': text
    })
    return result.inserted_id

def get_post_with_comments(post_id):
    post = posts.find_one({'_id': post_id})
    comments_ = list(comments.find({'post_id': post_id}))
    return dict(post=post, comments=comments_)

In [ ]:
post_id = make_post(
    title='First post', author='Rick', text='My very first post')
post_id
comment_id = make_comment(
    post_id=post_id,
    author='Nancy',
    text='This is an uninteresting post')

In [ ]:
cli_mongos.admin.command('enableSharding', 'test')

# Routeable, but not balanced

In [ ]:
comments.create_index('post_id')

In [ ]:
cli_mongos.admin.command(
    'shardCollection', 'test.comments', 
    key={'post_id': 1})

In [ ]:
cli_mongos.config.collection_names()

In [ ]:
list(cli_mongos.config.chunks.find())

In [ ]:
get_post_with_comments(post_id)

# Routeable and balanced

In [ ]:
db.comments.drop()

In [ ]:
comments.create_index([('post_id', 'hashed')])

In [ ]:
cli_mongos.admin.command(
    'shardCollection', 'test.comments', 
    key={'post_id': 'hashed'})

In [ ]:
make_comment(
    post_id=post_id,
    author='Nancy',
    text='This is an uninteresting post')

In [ ]:
get_post_with_comments(post_id)

In [ ]:
list(cli_mongos.config.chunks.find())